![NYC Skyline](nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many Airbnb listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this project, we will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx`.

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
This is a CSV file containing data on Airbnb listing prices and locations.
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed

In [141]:
# Import necessary packages
import pandas as pd
import numpy as np

## Load different datasets

In [142]:
airbnb_price_df = pd.read_csv('data/airbnb_price.csv')
print(airbnb_price_df.head())

   listing_id        price                nbhood_full
0        2595  225 dollars         Manhattan, Midtown
1        3831   89 dollars     Brooklyn, Clinton Hill
2        5099  200 dollars     Manhattan, Murray Hill
3        5178   79 dollars  Manhattan, Hell's Kitchen
4        5238  150 dollars       Manhattan, Chinatown


In [143]:
airbnb_last_review_df = pd.read_csv('data/airbnb_last_review.tsv', delimiter='\t')
print(airbnb_last_review_df.head())

   listing_id    host_name   last_review
0        2595     Jennifer   May 21 2019
1        3831  LisaRoxanne  July 05 2019
2        5099        Chris  June 22 2019
3        5178     Shunichi  June 24 2019
4        5238          Ben  June 09 2019


In [144]:
excel_file_path = 'data/airbnb_room_type.xlsx'
data = pd.ExcelFile(excel_file_path)
print(data.sheet_names)

['airbnb_room_type']


In [145]:
airbnb_room_type_df = data.parse(0)
print(airbnb_room_type_df.head())

   listing_id                                description        room_type
0        2595                      Skylit Midtown Castle  Entire home/apt
1        3831            Cozy Entire Floor of Brownstone  Entire home/apt
2        5099  Large Cozy 1 BR Apartment In Midtown East  Entire home/apt
3        5178            Large Furnished Room Near B'way     private room
4        5238         Cute & Cozy Lower East Side 1 bdrm  Entire home/apt


## Combine Datasets into one

In [146]:
merged_df = airbnb_price_df.merge(airbnb_last_review_df, on='listing_id', how='inner') \
               .merge(airbnb_room_type_df, on='listing_id', how='inner')
print(merged_df.head())

   listing_id  ...        room_type
0        2595  ...  Entire home/apt
1        3831  ...  Entire home/apt
2        5099  ...  Entire home/apt
3        5178  ...     private room
4        5238  ...  Entire home/apt

[5 rows x 7 columns]


## Inspect new merged data frame and data types

In [147]:
print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25209 entries, 0 to 25208
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   price        25209 non-null  object
 2   nbhood_full  25209 non-null  object
 3   host_name    25201 non-null  object
 4   last_review  25209 non-null  object
 5   description  25199 non-null  object
 6   room_type    25209 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.5+ MB
None


We can see price is of data type object instead of float. Last review is also not date and room type should be categorical. Let us work on those points. Also there are a few missing values but they don't affect our case study so it is okay to keep them as they are.

In [148]:
merged_df['price'] = merged_df['price'].str.split().str[0].astype(int)
print(merged_df['price'])

0        225
1         89
2        200
3         79
4        150
        ... 
25204    129
25205     45
25206    235
25207    100
25208     30
Name: price, Length: 25209, dtype: int64


In [149]:
merged_df['last_review'] = pd.to_datetime(merged_df['last_review'])
print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25209 entries, 0 to 25208
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   listing_id   25209 non-null  int64         
 1   price        25209 non-null  int64         
 2   nbhood_full  25209 non-null  object        
 3   host_name    25201 non-null  object        
 4   last_review  25209 non-null  datetime64[ns]
 5   description  25199 non-null  object        
 6   room_type    25209 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 1.5+ MB
None


In [150]:
print(merged_df['last_review'])

0       2019-05-21
1       2019-07-05
2       2019-06-22
3       2019-06-24
4       2019-06-09
           ...    
25204   2019-07-07
25205   2019-07-07
25206   2019-07-07
25207   2019-07-07
25208   2019-07-08
Name: last_review, Length: 25209, dtype: datetime64[ns]


In [151]:
print(merged_df['room_type'].unique())

['Entire home/apt' 'private room' 'Private room' 'entire home/apt'
 'PRIVATE ROOM' 'shared room' 'ENTIRE HOME/APT' 'Shared room'
 'SHARED ROOM']


In [152]:
merged_df['room_type'] = merged_df['room_type'].str.lower().astype('category')
print(merged_df['room_type'].cat.categories)

Index(['entire home/apt', 'private room', 'shared room'], dtype='object')


In [153]:
print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25209 entries, 0 to 25208
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   listing_id   25209 non-null  int64         
 1   price        25209 non-null  int64         
 2   nbhood_full  25209 non-null  object        
 3   host_name    25201 non-null  object        
 4   last_review  25209 non-null  datetime64[ns]
 5   description  25199 non-null  object        
 6   room_type    25209 non-null  category      
dtypes: category(1), datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB
None


## Let us start tackling the problems at hand

In [154]:
# What are the dates of the earliest and most recent reviews? Store these values as two separate variables with your preferred names.
earliest_review_date = merged_df['last_review'].min()
most_recent_review_date = merged_df['last_review'].max()
print(earliest_review_date)
print(most_recent_review_date)

2019-01-01 00:00:00
2019-07-09 00:00:00


In [155]:
# How many of the listings are private rooms? Save this into any variable.
num_private_rooms_listings = merged_df[merged_df['room_type'] == 'private room'].shape[0]
print(num_private_rooms_listings)

11356


In [156]:
# What is the average listing price? Round to the nearest two decimal places and save into a variable.
avg_price = round(merged_df['price'].mean(), 2)
print(avg_price)

141.78


In [157]:
# Combine the new variables into one DataFrame called review_dates with four columns in the following order: first_reviewed, last_reviewed, nb_private_rooms, and avg_price. The DataFrame should only contain one row of values.
review_dates = pd.DataFrame({
    'first_reviewed': [earliest_review_date],
    'last_reviewed': [most_recent_review_date],
    'nb_private_rooms': [num_private_rooms_listings],
    'avg_price': [avg_price]
})
print(review_dates.head())

  first_reviewed last_reviewed  nb_private_rooms  avg_price
0     2019-01-01    2019-07-09             11356     141.78
